In [ ]:
import numpy as np
import gc
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import pickle
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve

2025-03-08 16:01:54.558235: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 16:01:54.572514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741424514.587801  368723 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741424514.592142  368723 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 16:01:54.609068: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
N_MFCC = 128  

In [ ]:
X = np.load('../SavedFeatures/X_mfcc.npy', allow_pickle=True)
y = np.load('../SavedFeatures/y_mfcc.npy', allow_pickle=True)


X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [66]:
def cus_CNN(input_shape, num_classes):
    model_input = Input(shape=input_shape) 
    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(model_input)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.5)(x)  
    
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)  

    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)  
    x = Flatten()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)  
    output = Dense(num_classes, activation='softmax')(x) 
    return Model(inputs=model_input, outputs=output)

In [69]:
cnnmodel = cus_CNN(input_shape=(N_MFCC, 109, 1), num_classes=2)

cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [70]:
tf.keras.backend.clear_session()

In [71]:
cnnmodel.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.6040 - loss: 1.7143 - val_accuracy: 0.7774 - val_loss: 0.6316
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6838 - loss: 0.6126 - val_accuracy: 0.8037 - val_loss: 0.6081
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7055 - loss: 0.5743 - val_accuracy: 0.8127 - val_loss: 0.5390
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7475 - loss: 0.5365 - val_accuracy: 0.8186 - val_loss: 0.4722
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7765 - loss: 0.5187 - val_accuracy: 0.8536 - val_loss: 0.4416
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8086 - loss: 0.4758 - val_accuracy: 0.8076 - val_loss: 0.4290
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8454 - loss: 0.4150 - val_accuracy: 0.8390 - val_loss: 0.3481
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8724 - loss: 0.3259 - val_accuracy: 0.

#### Evaluation Metrics

In [ ]:
test_loss, test_accuracy = cnnmodel.evaluate(X_val, y_val, verbose=0)
y_pred = cnnmodel.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_val.argmax(axis=1) 


f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================CNN - MFCC===================")
print(f"Accuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")

# Tính EER
eers = []
for i in range(y_pred.shape[1]):
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")
# Average EER across classes
mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
===================CNN - MFCC===================
Accuracy: 0.92540
F1-Score: 0.92394
Class 0: EER = 0.06360 at threshold 0.96191
Class 1: EER = 0.06230 at threshold 0.03833
EER: 0.06295


In [73]:
# Save model as H5 File
cnnmodel.save("../SavedModels/MFCC_cus_cnn.h5")

In [ ]:
del cnnmodel, X_train, X_val, y_train, y_val  
gc.collect()